# 1. Introduction

Phase 2 - Milestone 3

Name    : Verren Monica

Batch   : RMT-038

URL Dataset Source  : [link](https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales/data)

In this notebook, I will perform data validation using one of the Python tools, Great Expectations (GX). The dataset used is sales data from Supermarket ABC for a 3-month period, which has already undergone data cleaning in the ETL process using Airflow.

# 2. Import Libraries

In [1]:
# Import Libraries
from great_expectations.data_context import FileDataContext
import pandas as pd

# 3. Create Data Context

In this section, a folder is initialized to store all configurations related to GX that will be created.

In [2]:
# Create data context
context = FileDataContext.create(project_root_dir='./')

# 4. Connect to Data Source

In this section, the connection configuration to the dataset that will be used will be set up.

In [3]:
# Give name to datasource
datasource_name = 'supermarket-sales'
datasource = context.sources.add_pandas(datasource_name)

# Give name to a data asset
asset_name = 'sales-jan-mar'
path_to_data = 'P2M3_verrenmonica_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# 5. Create Expectation Suite

In this section, an Expectation Suite is created to group validation rules into a single unit. Additionally, a validator will be created to access the data so that validation can be performed directly

In [4]:
# Creat an expectation suite
expectation_suite_name = 'expectation-supermarket-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,total,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37:00,Ewallet,604.17,4.761905,30.2085,5.3


# 6. Data Validation

In this section, data validation will be performed using Great Expectations. Some of the validations that will be carried out are:

1. column values to be unique
2. column values to be between min_value and max_value
3. column values to be in set
4. column values to be in type list
5. table column count to equal
6. column value lengths to equal
7. table columns to match ordered list

## 6.1. Expectation 1 - column values to be unique

In this section, data validation will be performed on the `invoice_id` column. This column contains invoice numbers, which serve as unique identifiers for each transaction. The uniqueness of the `invoice_id` column is important for Supermarket ABC to easily track information related to each transaction. Therefore, data validation will be done by ensuring that each value in the `invoice_id` column is unique.

In [5]:
# Expectation 1: Column `invoice_id` must be unique
validator.expect_column_values_to_be_unique('invoice_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

From the inspection results above, a success result of `true` was obtained. This means that the validation was successful, and the values in the `invoice_id` column are all unique, with no duplicates.

## 6.2. Expectation 2 - column values to be between min_value and max_value

In this section, validation will be performed on the `rating` column. This column contains customer ratings for their overall shopping experience at Supermarket ABC. The values in this column are expressed on a scale from 1 to 10, where 1 indicates a very poor experience and 10 indicates an excellent experience. Therefore, data validation will be performed to ensure that the values in the `rating` column are between the minimum value of 1 and the maximum value of 10.

In [6]:
# Expectation 2: Column `rating` must be between 1 and 10
validator.expect_column_values_to_be_between(column='rating', min_value=1, max_value=10)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

From the inspection results above, a success result of true was obtained. This means that the validation was successful, and the values in the rating column are within the scale of 1 to 10.

## 6.3. Expectation 3 - column values to be in set

In this section, validation will be performed on the `product_line` column. This column contains the product categories sold at Supermarket ABC, which include:
1. Electronic accessories
2. Fashion accessories
3. Food and beverages
4. Health and beauty
5. Home and lifestyle
6. Sports and travel

Therefore, validation will be performed to ensure that the values in the `product_line` column are one of these categories (to be in the set).

In [7]:
# Expectation 3: Column `product_line` must contain one of the product category
validator.expect_column_values_to_be_in_set('product_line', ["Electronic accessories",
                                                             "Fashion accessories",
                                                             "Food and beverages",
                                                             "Health and beauty",
                                                             "Home and lifestyle",
                                                             "Sports and travel"])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

From the inspection results above, a success result of `true` was obtained. This means that the validation was successful, and the values in the `product_line` column are all one of the categories of products sold by Supermarket ABC.

## 6.4. Expectation 4 - column values to be in type list

In this section, data validation will be performed on the `total` column. This column contains the total amount to be paid by the customer in a transaction, including taxes. The values in this column should be of type integer or float, as the `total` column is necessary for accurate data processing and analysis, such as total revenue and transaction analysis. Therefore, validation will be performed to ensure that the values in the `total` column are of type integer or float.

In [8]:
# Expectation 4: Column `total` must in form of integer or float
validator.expect_column_values_to_be_in_type_list('total', ['int64', 'float64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "result": {
    "observed_value": "float64"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

From the inspection results above, a success result of `true` was obtained. This means that the validation was successful, and it can be seen that the `total` column is of type float64.

## 6.5. Expectation 5 - table column count to equal

In this section, validation of the number of columns in the Supermarket ABC sales dataset will be performed. This validation aims to ensure that the data, which has passed through the cleaning process in the ETL pipeline (Airflow), maintains a consistent column structure. In other words, this validation is to detect any unwanted changes in the number of columns that may occur during the ETL process.

Before the data enters the ETL process, the Supermarket ABC sales dataset has 17 columns. Therefore, in this step, a check will be performed to ensure that the number of columns after the cleaning process matches the number of columns before the cleaning process, which is 17 columns. This is important to ensure that no columns are missing or duplicated, which could indicate an issue in the ETL process.

In [9]:
# Expectation 5: Number of column must be equal to 17
validator.expect_table_column_count_to_equal(17)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "result": {
    "observed_value": 17
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

From the inspection results above, a success result of `true` was obtained. This means that the validation was successful, and it can be seen that the number of columns in the dataset after the cleaning process is the same as before the cleaning process, which is 17 columns.

## 6.6. Expectation 6 - column value lengths to equal

In this section, data validation will be performed on the `invoice_id` column. The goal is to ensure the consistency of the `invoice_id` format in each transaction so that data consistency is maintained and integration with other systems is facilitated. From the Supermarket ABC sales dataset, it is known that `invoice_id` has 11 characters.

In [10]:
# Expectation 6: value lengthin 'invoice_id' must be equal to 11
validator.expect_column_value_lengths_to_equal('invoice_id', 11)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

From the inspection results above, a success result of `true` was obtained. This means that the validation was successful, and all values in the `invoice_id` column have a length of 11 characters."id memiliki panjang 11 karakter

## 6.7. Expectation 7 - table columns to match ordered list

In this section, validation will be performed on the column names and their order in the dataset. During the cleaning process in ETL, the column names were modified to align with the desired standards. Therefore, this validation aims to ensure that the column names and their order after the ETL process match the expected specifications.

In [11]:
# Expectation 7: the columns in the table exactly match specified list
validator.expect_table_columns_to_match_ordered_list(
    column_list=['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'unit_price', 'quantity', 'tax_5%', 'total', 'date',
       'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income',
       'rating']
)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "result": {
    "observed_value": [
      "invoice_id",
      "branch",
      "city",
      "customer_type",
      "gender",
      "product_line",
      "unit_price",
      "quantity",
      "tax_5%",
      "total",
      "date",
      "time",
      "payment",
      "cogs",
      "gross_margin_percentage",
      "gross_income",
      "rating"
    ]
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

From the inspection results above, a success result of `true` was obtained. This means that the validation was successful, and the column names are in line with the expected specifications.

# 7. Save Expectations

In this section, all the expectations that have been defined will be saved into the Expectation Suite. This is done to store the validation configurations, ensure complete validation documentation, and facilitate automation and repetition.

In [12]:
# Save into Expectation Suite
validator.save_expectation_suite(discard_failed_expectations=False)

# 8. Create Checkpoint

In this section, a checkpoint will be created with the goal of running data validation using the Expectation Suite that has already been created and generating a validation report.

In [13]:
# Create a checkpoint
checkpoint = context.add_or_update_checkpoint(
    name = 'checkpoint',
    validator = validator,
)

In [14]:
# Run a checkpoint
checkpointResult = checkpoint.run()

Calculating Metrics:   0%|          | 0/31 [00:00<?, ?it/s]

# 9. Build Data Docs

In this section, a documentation page will be created, which is automatically generated by Great Expectations, to provide a summary of the validation results, the structure of the dataset configuration, and details of the expectations that passed and failed. This documentation can simplify data analysis and the audit process.

In [15]:
# Build data docs
context.build_data_docs()

{'local_site': 'file://d:\\Hacktiv8\\M3\\p2-ftds038-rmt-m3-verrenmo\\gx\\uncommitted/data_docs/local_site/index.html'}